In [2]:
cd(@__DIR__)
using Pkg; Pkg.activate("."); Pkg.instantiate()

using Revise

using DataFrames, Dates, Statistics
using DelimitedFiles, CSV

using OrdinaryDiffEq, DiffEqFlux, Flux
using SciMLSensitivity

using Optimization, BlackBoxOptim
using Zygote

using Interpolations

using Random
Random.seed!(123)


  Activating project at `~/Documenti/HydroNODE`


TaskLocalRNG()

In [22]:
includet("ae.jl")
data_path = joinpath(pwd(),"basin_dataset_public_v1p2")
data_x, data_y, timepoints = prepare_multiple_basins(data_path)

([8.381253 0.0 -11.42; 8.403639 0.0 -10.7; … ; 11.520014 0.0 11.41; 11.424003 0.32 14.075;;;; 8.662769 0.0 -5.655; 8.695014 0.0 -4.54; … ; 11.520002 0.0 14.085; 11.520002 0.0 14.65;;;; 8.542522 0.0 -8.41; 8.56005 0.0 -7.675; … ; 11.520011 0.14 13.24; 11.498417 0.0 14.66;;;; … ;;;; 0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0;;;; 0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0;;;; 0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0], [0.7090447; 0.692807; … ; 0.46006715; 0.42759183;;;; 1.6444391; 1.4570979; … ; 0.6785913; 0.699407;;;; 1.7327809; 1.5061865; … ; 0.09930167; 0.15061864;;;; … ;;;; 1.013314f-39; 0.0; … ; 1.014784f-39; 0.0;;;; 1.014785f-39; 0.0; … ; 1.013296f-39; 0.0;;;; 1.013297f-39; 0.0; … ; 1.014767f-39; 0.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0  …  2090.0, 2091.0, 2092.0, 2093.0, 2094.0, 2095.0, 2096.0, 2097.0, 2098.0, 2099.0])

In [38]:
includet("HydroNODE_data.jl")
includet("HydroNODE_models.jl")
includet("HydroNODE_training.jl")


ERROR: UndefVarError: chosen_model_id not defined
Stacktrace:
 [1] top-level scope
   @ ~/Documenti/HydroNODE/HydroNODE_models.jl:190
in expression starting at /home/alberto/Documenti/HydroNODE/HydroNODE_models.jl:190


In [23]:

function Encoder(enc_space_dim; drop_p=0.5)
    # Define the encoder and decoder networks
    encoder_features = Chain(
        Conv((4, 1), 1 => 8; stride = (4,1), pad = 0),
        BatchNorm(8, leakyrelu),
        Dropout(drop_p),
        
        Conv((4, 1), 8 => 8; stride = 1, pad = 0),
        BatchNorm(8, leakyrelu),
        Dropout(drop_p),
        MaxPool((2, 1), pad=0),
        
        Conv((6, 1), 8 => 16; stride = 1, pad = 0),
        BatchNorm(16, leakyrelu),
        Dropout(drop_p),
        
        Conv((2, 1), 16 => 16; stride = (2,1), pad = 0),
        BatchNorm(16, leakyrelu),
        Dropout(drop_p),
        MaxPool((2, 1), pad=0),
        
        Flux.flatten,
        Dense(16 * 64 * 1, 256, relu),
        Dense(256, enc_space_dim, relu)
    ) |> gpu
    
    return encoder_features
end



Encoder (generic function with 1 method)

In [25]:
loader = Flux.DataLoader((data_x[:,:,:,1:560], data_y[:,:,:,1:560]), batchsize=16, shuffle=true)
print(sizeSEQ_LEN(data_x))
print(size(data_y))

(2100, 3, 1, 598)(2100, 1, 1, 598)

In [37]:
# define the encoder network
enc_space_dim = 27
enc = Encoder(enc_space_dim)

# choose optimizer
optimizer = ADAM(0.001)

seq_len = size(timepoints)[1]

# training loop
num_epochs = 1
for epoch in 1:num_epochs
    for batch in loader
        # retrieve batches
        batch_x, batch_y = batch
        # compute encded features
        enc_data = enc(batch_y)
        # unqueeze enc_data
        enc_data = Flux.unsqueeze(Flux.unsqueeze(enc_data, 2), 1)
        extended_data = cat(dims=2, batch_x, repeat(enc_data, seq_len, 1, 1, 1))
        print(size(extended_data))
    end
end


(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)(2100, 10, 1, 16)

In [ ]:
a = ["2", "3", "4", "5"]
ind = [1,3]
a[ind]

In [35]:
function ConvEncoder(; imgsize=(365,1,1), nclasses=27) 
    out_conv_size = (imgsize[1]÷4 - 3, 1, 16)
    
    return Chain(
            Conv((5, 1), imgsize[end]=>8, relu),
            MaxPool((2, 1)),
            Conv((5, 1), 6=>16, relu),
            MaxPool((2, 1)),
            flatten,
            Dense(prod(out_conv_size), 128, relu), 
            Dense(128, 64, relu), 
            Dense(64, nclasses)
          )
end

ConvEncoder (generic function with 1 method)